# Transformerの実装(分類タスク用)

In [1]:
#パッケージのインポート
import glob
import os
import os.path as osp
import random
import numpy as np
import json
from PIL import Image
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import torch.optim as optim
import torch.utils.data as data
#import torchvision
#from torchvision import models,transforms
import cv2 
import time
import math
from matplotlib import cm

import gensim
import torchtext
#import spacy

## Embedderモジュール

In [2]:
class Embedder(nn.Module):
    """ idで示されている単語をベクトルに変換します """ 
    def __init__(self,text_embedding_vectors):
        super(Embedder,self).__init__()

        self.embeddings=nn.Embedding.from_pretrained(embeddings=text_embedding_vectors, freeze=True)

    def forward(self,x):
        x_vec=self.embeddings(x)

        return x_vec

In [23]:
os.path

<module 'posixpath' from '/Users/kawadatakeshiho/opt/anaconda3/envs/pytorchenv/lib/python3.7/posixpath.py'>

In [24]:
sys.path

['/Users/kawadatakeshiho/Desktop/pytorch_advanced/syakyo',
 '/Users/kawadatakeshiho/Desktop/pytorch_advanced/7_nlp_sentiment_transformer',
 '/Users/kawadatakeshiho/.vscode/extensions/ms-toolsai.jupyter-2022.9.1202862440/pythonFiles',
 '/Users/kawadatakeshiho/.vscode/extensions/ms-toolsai.jupyter-2022.9.1202862440/pythonFiles/lib/python',
 '/Users/kawadatakeshiho/opt/anaconda3/envs/pytorchenv/lib/python37.zip',
 '/Users/kawadatakeshiho/opt/anaconda3/envs/pytorchenv/lib/python3.7',
 '/Users/kawadatakeshiho/opt/anaconda3/envs/pytorchenv/lib/python3.7/lib-dynload',
 '',
 '/Users/kawadatakeshiho/opt/anaconda3/envs/pytorchenv/lib/python3.7/site-packages',
 '/Users/kawadatakeshiho/opt/anaconda3/envs/pytorchenv/lib/python3.7/site-packages/IPython/extensions',
 '/Users/kawadatakeshiho/.ipython']

#### 前回分

In [24]:
import glob
import os
import io
import string

import re
import random
#import spacy
import torchtext
from torchtext.vocab import Vectors

In [26]:
def preprocessing_text(text):
        # 改行コードを消去
        text = re.sub('<br />', '', text)

        # カンマ、ピリオド以外の記号をスペースに置換
        for p in string.punctuation:
            if (p == ".") or (p == ","):
                continue
            else:
                text = text.replace(p, " ")

        # ピリオドなどの前後にはスペースを入れておく
        text = text.replace(".", " . ")
        text = text.replace(",", " , ")
        return text

# 分かち書き（今回はデータが英語で、簡易的にスペースで区切る）
def tokenizer_punctuation(text):
    return text.strip().split()

# 前処理と分かち書きをまとめた関数を定義
def tokenizer_with_preprocessing(text):
        text = preprocessing_text(text)
        ret = tokenizer_punctuation(text)
        return ret


In [29]:
def get_IMDb_DataLoaders_and_TEXT(max_length=256, batch_size=24):
    """IMDbのDataLoaderとTEXTオブジェクトを取得する。 """
    # データを読み込んだときに、読み込んだ内容に対して行う処理を定義します
    # max_length
    TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer_with_preprocessing, use_vocab=True,lower=True, include_lengths=True, batch_first=True, fix_length=max_length, init_token="<cls>", eos_token="<eos>")
    LABEL = torchtext.data.Field(sequential=False, use_vocab=False)

    # フォルダ「data」から各tsvファイルを読み込みます
    train_val_ds, test_ds = torchtext.data.TabularDataset.splits(path='./data/', train='IMDb_train.tsv',test='IMDb_test.tsv', format='tsv',fields=[('Text', TEXT), ('Label', LABEL)])

    # torchtext.data.Datasetのsplit関数で訓練データとvalidationデータを分ける
    train_ds, val_ds = train_val_ds.split(split_ratio=0.8, random_state=random.seed(1234))

    # torchtextで単語ベクトルとして英語学習済みモデルを読み込みます
    english_fasttext_vectors = Vectors(name='./data/wiki-news-300d-1M.vec')

    # ベクトル化したバージョンのボキャブラリーを作成します
    TEXT.build_vocab(train_ds, vectors=english_fasttext_vectors, min_freq=10)

    # DataLoaderを作成します（torchtextの文脈では単純にiteraterと呼ばれています）
    train_dl = torchtext.data.Iterator(train_ds, batch_size=batch_size, train=True)

    val_dl = torchtext.data.Iterator(val_ds, batch_size=batch_size, train=False, sort=False)

    test_dl = torchtext.data.Iterator(test_ds, batch_size=batch_size, train=False, sort=False)

    return train_dl, val_dl, test_dl, TEXT


### 　前回分終わり
###  動作確認

In [30]:
#動作確認
#前節のDataLoaderなどを取得
#from utils.fixing_dataloader import get_IMDb_DataLoaders_and_TEXT
train_dl,val_dl,test_dl,TEXT=get_IMDb_DataLoaders_and_TEXT(max_length=256,batch_size=24)

In [31]:
#ミニバッチの用意
batch=next(iter(train_dl))

#モデル構築
net1=Embedder(TEXT.vocab.vectors)

#入出力
x=batch.Text[0]
x1=net1(x) #単語をベクトルに

print("入力のテンソルサイズ：",x.shape)
print("出力のテンソルサイズ：",x1.shape)

入力のテンソルサイズ： torch.Size([24, 256])
出力のテンソルサイズ： torch.Size([24, 256, 300])


In [32]:
x1

tensor([[[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [-0.3116,  0.0856, -0.0069,  ...,  0.0877,  0.1019,  0.0097],
         [-0.0303,  0.0524, -0.0671,  ...,  0.2191, -0.1200,  0.0505],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0047,  0.0223, -0.0087,  ...,  0.1479,  0.1324, -0.0318],
         [-0.1289,  0.0115,  0.0033,  ...,  0.1811,  0.0398, -0.0530],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0047,  0.0223, -0.0087,  ...,  0

## PositionalEncoderモジュール

In [34]:
class PositionalEncoder(nn.Module):
    """ 入力された単語の位置を示すベクトル情報を付加する """

    def __init__(self,d_model=300,max_seq_len=256):
        super().__init__()

        self.d_model=d_model #単語ベクトルの次元数

        #単語の順番(pos)と埋め込みベクトルの次元の位置(i)によって一意に定まる値の表をpeとして作成
        pe=torch.zeros(max_seq_len,d_model)

        #GPU使える場合はGPUへ送る、ここでは省略。実際に学習時には使用する
        for pos in range(max_seq_len):
            for i in range(0,d_model,2):
                pe[pos,i]=math.sin(pos/(10000**((2*i)/d_model)))
                pe[pos,i+1]=math.cos(pos/(10000**((2*(i+1))/d_model)))

        #表peの先頭に、ミニバッチ次元となる次元をたす
        self.pe=pe.unsqueeze(0)

        #勾配を計算しないようにする
        self.pe.requires_grad=False

    def forward(self,x):
        #入力xとPositional Encodingを足し算する
        #xがpeよりも小さいので、大きくする
        ret=math.sqrt(self.d_model)*x+self.pe
        return ret

In [36]:
#動作確認

#モデル構築
net1=Embedder(TEXT.vocab.vectors)
net2=PositionalEncoder(d_model=300,max_seq_len=256)

#入出力
x=batch.Text[0]
x1=net1(x) #単語をベクトルに
x2=net2(x1)

print("入力のテンソルサイズ：",x1.shape)
print("出力のテンソルサイズ：",x2.shape)

入力のテンソルサイズ： torch.Size([24, 256, 300])
出力のテンソルサイズ： torch.Size([24, 256, 300])


## TransformerBlockモジュール

In [37]:
class Attention(nn.Module):
    """ Transformerは本当はマルチヘッドAttentionですが、わかりやすさを優先しシングルAttentionで実装 """

    def __init__(self,d_model=300):
        super().__init__()

        #SAGANではadConvを使用したが、今回は全結合層で特徴量を変換する
        self.q_linear=nn.Linear(d_model,d_model)
        self.v_linear=nn.Linear(d_model,d_model)
        self.k_linear=nn.Linear(d_model,d_model)

        #出力時に使用する全結合層
        self.out=nn.Linear(d_model,d_model)

        #Attentionの大きさ調節の変数
        self.d_k=d_model

    def forward(self,q,k,v,mask):
        #全結合層で特徴量を変換
        k=self.k_linear(k)
        q=self.q_linear(q)
        v=self.v_linear(v)

        #Attentionの値を計算する
        #各値を足し算すると大きくなりすぎるので、root(d_k)でわって調整
        weights=torch.matmul(q,k.transpose(1,2))/math.sqrt(self.d_k)

        #ここでmaskを計算
        mask=mask.unsqueeze(1)
        weights=weights.masked_fill(mask==0, -1e9)

        #softmaxで規格化をする
        normalized_weights=F.softmax(weights,dim=-1)

        #AttentionとValueを掛け算
        output=torch.matmul(normalized_weights,v)

        #全結合層で特徴量を変換
        output=self.out(output)

        return output,normalized_weights



In [39]:
class FeedForward(nn.Module):
    def __init__(self,d_model,d_ff=1024,dropout=0.1):
        """ Attention層から出力を単純に全結合層2つで特徴量に変換するだけのユニットです """
        super().__init__()

        self.linear_1=nn.Linear(d_model,d_ff)
        self.dropout=nn.Dropout(dropout)
        self.linear_2=nn.Linear(d_ff,d_model)

    def forward(self,x):
        x=self.linear_1(x)
        x=self.dropout(F.relu(x))
        x=self.linear_2(x)
        return x

In [40]:
class TransformerBlock(nn.Module):
    def __init__(self,d_model,dropout=0.1):
        super().__init__()

        #LayerNormalization層
        self.norm_1=nn.LayerNorm(d_model)
        self.norm_2=nn.LayerNorm(d_model)

        #Attention層
        self.attn=Attention(d_model)

        #Attentionの後の全結合層2つ
        self.ff=FeedForward(d_model)

        #Dropout
        self.dropout_1=nn.Dropout(dropout)
        self.dropout_2=nn.Dropout(dropout)

    
    def forward(self,x,mask):
        #正規化とAttention
        x_normalized=self.norm_1(x)
        output,normalized_weights=self.attn(x_normalized,x_normalized,x_normalized,mask)

        x2=x+self.dropout_1(output)

        #正規化と全結合層
        x_normalized2=self.norm_2(x2)
        output=x2+self.dropout_2(self.ff(x_normalized2))

        return output,normalized_weights


In [41]:
#動作確認

#モデル確認
net1=Embedder(TEXT.vocab.vectors)
net2=PositionalEncoder(d_model=300,max_seq_len=256)
net3=TransformerBlock(d_model=300)

#maskの作成
x=batch.Text[0]
input_pad=1 #単語IDにおいて<pad>:1のため
input_mask=(x!=input_pad)
print(input_mask[0])

#入出力
x1=net1(x) #単語をベクトルに
x2=net2(x1) #Position情報を足し算
x3,normlizes_weights=net3(x2,input_mask) #Self-Attentionで特徴量を変換

print("入力のテンソルサイズ：",x2.shape)
print("出力のテンソルサイズ：",x3.shape)
print("Attentionのサイズ:",normlizes_weights.shape)

tensor([ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True, False, False, 

## ClassificationHeadモジュール

In [48]:
class ClassificationHead(nn.Module):
    """ Transformer_Blockの出力を使用し、最後のクラス分類をさせる """
    def __init__(self,d_model=300,output_dim=2):
        super().__init__()

        #全結合層
        self.linear=nn.Linear(d_model,output_dim) #output_dimはポジ・ネガの2つ

        #重み初期化処理
        nn.init.normal_(self.linear.weight,std=0.02)
        nn.init.normal_(self.linear.bias,0)

    def forward(self,x):
        x0=x[:,0,:] #各ミニバッチの各文の先頭の単語の特徴量(300次元)を取り出す
        out=self.linear(x0)

        return out

## Transformerの実装

In [49]:
#最終的なTransformerモデルのクラス

class TransformerClassification(nn.Module):
    """ Transformerでクラス分類させる """

    def __init__(self,text_embedding_vectors,d_model=300,max_seq_len=256,output_dim=2):
        super().__init__()

        #モデル構築
        self.net1=Embedder(text_embedding_vectors)
        self.net2=PositionalEncoder(d_model=d_model,max_seq_len=max_seq_len)
        self.net3_1=TransformerBlock(d_model=d_model)
        self.net3_2=TransformerBlock(d_model=d_model)
        self.net4=ClassificationHead(output_dim=output_dim,d_model=d_model)

    def forward(self,x,mask):
        x1=self.net1(x) #単語をベクトルに
        x2=self.net2(x1) #Position情報を足し算
        x3_1,normlizes_weights_1=self.net3_1(x2,mask) #Self-Attentionで特徴量を変換
        x3_2,normalizes_weights_2=self.net3_2(x3_1,mask) #Self-Attentionで特徴量を変換
        x4=self.net4(x3_2)

        return x4,normlizes_weights_1,normalizes_weights_2

In [50]:
#動作確認
#ミニバッチの用意
batch=next(iter(train_dl))

#モデル構築
net=TransformerClassification(
    text_embedding_vectors=TEXT.vocab.vectors, d_model=300, max_seq_len=256, output_dim=2)

#入出力
x=batch.Text[0]
input_mask=(x!=input_pad)
out,normlizes_weights_1,normlizes_weights_2=net(x,input_mask)

print("出力のテンソルサイズ：",out.shape)
print("出力テンソルのsigmoid：",F.softmax(out,dim=1))

出力のテンソルサイズ： torch.Size([24, 2])
出力テンソルのsigmoid： tensor([[0.2452, 0.7548],
        [0.2741, 0.7259],
        [0.2847, 0.7153],
        [0.2676, 0.7324],
        [0.2755, 0.7245],
        [0.2989, 0.7011],
        [0.2412, 0.7588],
        [0.2641, 0.7359],
        [0.2555, 0.7445],
        [0.2290, 0.7710],
        [0.2819, 0.7181],
        [0.2626, 0.7374],
        [0.2771, 0.7229],
        [0.2367, 0.7633],
        [0.2540, 0.7460],
        [0.2520, 0.7480],
        [0.2669, 0.7331],
        [0.2418, 0.7582],
        [0.3119, 0.6881],
        [0.2459, 0.7541],
        [0.2456, 0.7544],
        [0.2988, 0.7012],
        [0.2252, 0.7748],
        [0.2257, 0.7743]], grad_fn=<SoftmaxBackward>)
